## 0. Introduction

The aim of this lab is to get familiar with **Image Classification** using  **KNN**, and **SVM** models.


1.   This lab is the first course-work activity **Assignment 1**
2.   A report answering the <font color = 'red'>**questions in</font><font color = "maroon"> red**</font> should be submitted on QMplus along with the completed Notebook.
3. The report should be a separate file in **pdf format** (so **NOT** *doc, docx, notebook* etc.), well identified with your name, student number, assignment number (for instance, Assignment 1), module code.
4. Make sure that **any figures or code** you comment on, are **included in the report**.
5. No other means of submission other than the appropriate QM+ link is acceptable at any time (so NO email attachments, etc.)
6. **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!).


In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.cluster import KMeans

import numpy as np
import cv2
from google.colab.patches import cv2_imshow

import glob
import os
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import random

Upload the `images.zip` file to colab and unzip using the cells below. Remember that the file upload is ephemeral, so the files are uploaded only for eac session.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/analysis")


In [ ]:
!unzip images.zip

The **image** folder contains 5 sub-directories each of which contains images from one of the following image classes: airplanes, cars, dog, faces and keyboard. In each class there are 80 images, the first 60 will be used for training and the rest will be used for testing.

An example of the images can be seen below:

In [ ]:
## example of 5 images from each class
example_img = list()
for root, dirs, _ in os.walk('image/'):
    for class_folder in dirs:
      img_files = os.listdir(os.path.join(root, class_folder))
      img_files = [
          os.path.join(*[root, class_folder, filename]) for filename in img_files
          ]
      example_img.extend(img_files[:5])

img_lst = list()
for imgf in example_img:
    img = cv2.imread(imgf)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_lst.append(img)

f, axarr = plt.subplots(5,5,figsize=(15, 15))
axarr = axarr.flatten()
for img, ax in zip(img_lst, axarr):
    ax.imshow(img)
plt.show()

SIFT (Scale Invariant Fourier Transform) Detector is used in the detection of interest points on an input image. It allows identification of localized features in images. The following script generates images with keypoints for the above examples.

In [ ]:
f, axarr = plt.subplots(5,5,figsize=(15, 15))
axarr = axarr.flatten()
for img, ax in zip(img_lst, axarr):
    # Converting image to grayscale
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Applying SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    kp = sift.detect(gray, None)
    # Marking the keypoint on the image using circles
    img=cv2.drawKeypoints(gray,kp,img,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    ax.imshow(img)
plt.show()

The first step is to compute the SIFT descriptors for all images in the data directory. We have precomputed the SIFT image descriptors for 900 predifined patches in each image. These can be imported from `all_features.mat` file.

In [ ]:
label = {'airplanes':0, 'cars':1, 'dog':2, 'faces':3, 'keyboard':4}

In [ ]:
import scipy.io

mat = scipy.io.loadmat('all_features.mat')
train_des = mat['TrainMat']
test_des = mat['TestMat']
train_label = list()
test_label = list()
for i in range(5):
    train_label.extend([i] * 60)
    test_label.extend([i] * 20)

In [ ]:
train_des = train_des.reshape((300, -1, 128)) # 300个图像
test_des = test_des.reshape((100, -1, 128))

## 2. Dictionary Creation - Feature Quantization

Task 1: Using [sklearn KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), create a dictionary by clustering a **subset** (eg. 6000) of the extracted descriptors.

Use a dictionary of 500 words and the `elkan` algorithm to compute the dictionary.

<font color = 'red'>Explain the input and the output of the K-means; what are the input and output dimensions? what is the use of the dictionary?</font> [3 marks]

In [ ]:
### YOUR CODE HERE
# Step 1: create an array (N x 128) of all descriptors in the training set (not by image)
all_descriptors = np.vstack(train_des) # 垂直堆叠在一起后，将所有图像的描述符堆叠成一个大的二维数组
subset_descriptors = all_descriptors[:6000, :]

# Step 2: use Kmeans to create the Dictionary (the resulting Dictionary should have K words and 128 features)
K = 500
kmeans = KMeans(n_clusters=K, algorithm='elkan', random_state=42)
kmeans.fit(subset_descriptors)
dictionary = kmeans.cluster_centers_

Task 2: Assign each image descriptor in the training and test sets, to the nearest codeword cluster. [2 marks]

In [ ]:
### YOUR CODE HERE
train_assignments = kmeans.predict(train_des.reshape(-1, 128))
test_assignments = kmeans.predict(test_des.reshape(-1, 128)) #输出一个与输入特征描述符数量相同的数组，数组中的每个元素是对应描述符被分配到的聚类中心的索引

train_reAssignments = train_assignments.reshape(train_des.shape[:-1]) # 将聚类分配的结果重新塑形为与原始特征描述符数组train_des和test_des在除最后一个维度外的形状相匹配。最后一个维度（在这个场景下是128，即每个描述符的长度）被忽略，因为聚类分配的结果是基于每个描述符而不是每个维度进行的
test_reAssignments = test_assignments.reshape(test_des.shape[:-1])

## 3. Image Representation using BoW
Task 3: Represent each image in the training and the test dataset as a histogram of visual words (i.e. represent each image using the Bag of Words representation). Normalise the histograms by their L1 norm. [2 marks]

In [ ]:
### YOUR CODE HERE
# Step 1: For each image, create a histogram of the descriptors
# i.e. a histogram of the allocated clusters
def create_histogram(assignments, K):
    histograms = np.zeros((assignments.shape[0], K))
    for i in range(assignments.shape[0]):
      hist, _ = np.histogram(assignments[i], bins=np.arange(K+1)) # 在np.histogram函数中，bins参数定义了直方图的分组依据，即你希望数据被分割成多少个区间。通过指定bins=np.arange(K+1)，你告诉函数使用一个包含从0到K（包含K）的整数序列作为边界来创建直方图的区间。
      histograms[i, :] = hist
    return histograms

# 每一行代表着训练数据集中的一个样本。
#每一列代表着一个聚类中心（或者说是一个聚类）。这里的 K 是聚类的数量，因此就有 K 个列。
# 每个单元格的值表示对应样本分配给对应聚类的次数（即直方图中每个聚类的频数）。
train_histograms = create_histogram(train_reAssignments, K)
test_histograms = create_histogram(test_reAssignments, K) # histograms是一个二维数组，每行代表一张图像的直方图，列数等于聚类的数量K

# Step 2: Normalize by the L1 norm of the vector
from sklearn.preprocessing import normalize

# 直方图通过L1范数进行归一化。L1范数归一化是指将直方图的每个元素除以直方图所有元素的绝对值之和，确保每个直方图向量的元素之和为1。这一步骤通常用于确保模型不会因为图像大小不同而受到影响，同时使特征向量更易于机器学习模型处理
train_histograms= normalize(train_histograms, norm='l1', axis=1)
test_histograms = normalize(test_histograms, norm='l1', axis=1)

## 4. Image Classification using a Nearest Neighbour Classifier
Task 4: Implement the Euclidean distance for the multi-dimensional case. Using sklearn [KNN classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) and the distance implemented  (passed using keyword argument `metric`, check KNN API for details), train a model on the train set BoW, for K=1 [3 marks]

In [ ]:
def euclidean_distance(hist_1: np.array, hist_2: np.array):
    """
      hist_1: a 1D vector representing a histogram
      hist_2: a 1D vector representing a histogram
      returns:
      The distance between two histograms (float)
    """
    dist = 0
    ### YOUR CODE HERE
    squared_diff = np.sum((hist_1 - hist_2) ** 2)
    dist = np.sqrt(squared_diff)

    return dist

### YOUR CODE HERE
knn_euclidean = KNeighborsClassifier(n_neighbors=1, metric=euclidean_distance)
knn_euclidean.fit(train_histograms, train_label)

Task 5: Implement a method for histogram intersection [2 marks]

In [ ]:
def hist_intersection(hist_1: np.array, hist_2: np.array):
  """
    hist_1: a 1D vector representing a histogram
    hist_2: a 1D vector representing a histogram
    returns:
    The distance between two histograms (float)
  """
  dist = 0.
  ### YOUR CODE HERE
  dist = np.sum(np.minimum(hist_1, hist_2))
  return dist

Task 6: Train a second classifier using the `hist_intersection()` as the distance metric . <font color = 'red'> Evaluate both classifiers on the test set (in terms of accuracy and confusion matrices) and discuss results.</font> [3 marks]

In [ ]:
### YOUR CODE HERE
from sklearn.metrics import accuracy_score, confusion_matrix

knn_intersection = KNeighborsClassifier(n_neighbors=1, metric=hist_intersection)
knn_intersection.fit(train_histograms, train_label)

test_pred_euclidean = knn_euclidean.predict(test_histograms)
test_pred_intersection = knn_intersection.predict(test_histograms)

accuracy_euclidean = accuracy_score(test_label, test_pred_euclidean)
accuracy_intersection = accuracy_score(test_label, test_pred_intersection)

confusion_euclidean = confusion_matrix(test_label, test_pred_euclidean)
confusion_intersection = confusion_matrix(test_label, test_pred_intersection)

print("Euclidean Distance  Accuracy:", accuracy_euclidean)
print("Euclidean Distance Confusion Matrix:\n", confusion_euclidean)
print("Histogram Intersection Accuracy:", accuracy_intersection)
print("Histogram Intersection Confusion Matrix:\n", confusion_intersection)

## 5. Dictionary size

Task 7: <font color = 'red'>  Repeat steps 1-6 using a very small dictionary size (eg. 5). Compute the accuracy and confusion matrices and discuss the drop in performance </font> [2 marks]

In [ ]:
### YOUR CODE HERE
K_small = 5
kmeans_small = KMeans(n_clusters=K_small, random_state=42)
kmeans_small.fit(subset_descriptors)
dictionary_small = kmeans_small.cluster_centers_

train_assignments_small = kmeans_small.predict(train_des.reshape(-1, 128))
test_assignments_small = kmeans_small.predict(test_des.reshape(-1, 128))

train_reAssignments_small = train_assignments_small.reshape(train_des.shape[:-1])
test_reAssignments_small = test_assignments_small.reshape(test_des.shape[:-1])


train_histograms_small=create_histogram(train_reAssignments_small, K)
test_histograms_small=create_histogram(test_reAssignments_small, K)
train_histograms_small= normalize(train_reAssignments_small, norm='l1', axis=1)
test_histograms_small = normalize(test_reAssignments_small, norm='l1', axis=1)


knn_euclidean_small = KNeighborsClassifier(n_neighbors=1, metric=euclidean_distance)
knn_euclidean_small.fit(train_histograms_small, train_label)

knn_intersection_small = KNeighborsClassifier(n_neighbors=1, metric=hist_intersection)
knn_intersection_small.fit(train_histograms_small, train_label)


test_pred_euclidean_small = knn_euclidean_small.predict(test_histograms_small)
test_pred_intersection_small = knn_intersection_small.predict(test_histograms_small)

accuracy_euclidean_small = accuracy_score(test_label, test_pred_euclidean_small)
accuracy_intersection_small = accuracy_score(test_label, test_pred_intersection_small)

confusion_euclidean_small = confusion_matrix(test_label, test_pred_euclidean_small)
confusion_intersection_small = confusion_matrix(test_label, test_pred_intersection_small)

print("Small Dictionary Size Euclidean Distance Accuracy:", accuracy_euclidean_small)
print("Small Dictionary Size Euclidean Distance Confusion Matrix:\n", confusion_euclidean_small)
print("Small Dictionary Size Histogram Intersection Accuracy:", accuracy_intersection_small)
print("Small Dictionary Size Histogram Intersection Confusion Matrix:\n", confusion_intersection_small)


## 6. Support Vector Machines
In this section we will train a linear [SVM classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).


Task 8: Using [Grid Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) select optimal hyperparameters `C` and `gamma`.

<font color = 'red'> Evaluate SVC classifier on the test set (in terms of accuracy and confusion matrices) and compare to the KNN classifier results. </font>

<font color = 'red'> For each class show some images that are correctly classified and some images that are incorrectly classified. </font>

<font color = 'red'> Can you explain some of the failures?</font> [3 marks]

In [ ]:
parameters = {'gamma':[2**x for x in np.arange(-1, 1.6, 0.1)], 'C':[2**x for x in range(1, 11)]} # gamma和C是支持向量机（SVC）中的两个重要的超参数。gamma参数定义了单个训练样本的影响范围，值越大表示样本的影响范围越小。C是正则化参数，用于控制误分类的惩罚程度，值越大表示允许的误分类数量越少

### YOUR CODE HERE

from sklearn.svm import SVC
#Grid Search是一种常用的超参数优化技术，它通过穷举搜索给定超参数的组合，然后评估每个组合的性能，最终选择表现最好的超参数组合。对于每一组超参数的组合，Grid Search使用交叉验证来评估模型的性能，通常采用k折交叉验证，其中数据被划分为k个子集，每次使用其中k-1个子集进行训练，然后在剩余的子集上进行验证。这样可以更准确地估计模型的性能，同时避免了对单一数据划分的依赖性。
svc = SVC()
grid_search = GridSearchCV(svc, parameters, cv=5, scoring='accuracy') # cv=5 意味着采用了5折交叉验证，即数据集被分成5个子集，其中4个用于训练，1个用于验证
grid_search.fit(train_histograms, train_label)

best_svc = grid_search.best_estimator_ # 一个已经在训练数据上训练好的支持向量机模型，它使用了在网格搜索过程中确定的最佳超参数组合
test_pred_svc = best_svc.predict(test_histograms)


accuracy_svc = accuracy_score(test_label, test_pred_svc)
confusion_svc = confusion_matrix(test_label, test_pred_svc)

print("SVC Classifier Accuracy:", accuracy_svc)
print("SVC Classifier Confusion Matrix:\n", confusion_svc)


correct_indices = np.where(test_pred_svc == test_label)[0] #np.where(test_pred_svc == test_label) 函数会返回满足条件的元素的索引，即预测正确的样本在 test_pred_svc 数组中的索引。
#[0] 表示取出返回的元组中的第一个元素，这是因为 np.where() 返回的是一个元组，其中包含了符合条件的元素索引，而在这个特定的情况下，我们只关心索引值。
incorrect_indices = np.where(test_pred_svc != test_label)[0]


example_img = list()
for root, dirs, _ in os.walk('image/'):
    for class_folder in dirs:
      img_files = os.listdir(os.path.join(root, class_folder))
      img_files = [
          os.path.join(*[root, class_folder, filename]) for filename in img_files
          ]
      example_img.extend(img_files[-20:])

test_images = list()
for imgf in example_img:
    img = cv2.imread(imgf)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        test_images.append(img)
    else:
        print(f"Failed to load image: {imgf}")  # Print the path of any image that fails to load


class_labels_arrary = [0, 1, 2, 3, 4]

def show_images_by_class(indices, class_label, img_lst, class_labels, title):
    print(f'{title} for class {class_label}:')
    class_indices = [i for i in indices if class_labels[i] == class_label]
    if len(class_indices) > 0:
        num_images_to_show = min(len(class_indices), 5)  # Show up to 5 images
        fig, axs = plt.subplots(1, num_images_to_show, figsize=(15, 3))
        if num_images_to_show == 1:
            axs = [axs]  # Make it iterable
        for idx, ax in enumerate(axs):
            if idx < len(class_indices):  # Check if index is within range
                img_index = class_indices[idx]
                if img_index < len(img_lst):  # Ensure img_index is valid for img_lst
                    ax.imshow(img_lst[img_index])
                    # Updated title to include predicted and ground truth labels
                    ax.set_title(f"Ground Truth: {class_label} \n Predicted: {test_pred_svc[img_index]}")
                    ax.axis('off')
                else:
                    print(f"Index {img_index} out of range for img_lst.")
            else:
                print(f"No more images to show for class {class_label}.")
        plt.show()
    else:
        print(f"No images found for class {class_label}.")

for class_label in class_labels_arrary:
    show_images_by_class(correct_indices, class_label, test_images, test_label, "Correctly Classified Images")
    show_images_by_class(incorrect_indices, class_label, test_images, test_label, "Incorrectly Classified Images")



## 7. Handing in

Create a folder that will contain:

• A .pdf report that contains the answers to the exercises
• The programs files

Create a .zip file and submit electronically on QMplus.

IMPORTANT: Plagiarism (copying from other students, or copying the work of others without proper referencing) is cheating, and will not be tolerated.

IF TWO “FOLDERS” ARE FOUND TO CONTAIN IDENTICAL MATERIAL, BOTH WILL BE GIVEN A MARK OF ZERO.
